<a href="https://colab.research.google.com/github/ogyogy/colaboratory/blob/main/ajagekar2020quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ajagekar2020quantum
[PDF](https://arxiv.org/ftp/arxiv/papers/1910/1910.13045.pdf)

In [1]:
pip install openjij pyqubo pulp

In [2]:
# モジュールのインポート
import pulp

In [3]:
# MILPモデルの初期化
prob = pulp.LpProblem('MILP', pulp.LpMinimize)

In [4]:
# ジョブの数
I = 5
# マシンの数
M = 3

In [5]:
# ジョブiをマシンmで行うのにかかる時間
P = [
     [8, 3, 6],     # ジョブ1
     [5, 3, 10],    # ジョブ2
     [9, 4, 6],     # ジョブ3
     [4, 9, 11],    # ジョブ4
     [9, 4, 4]      # ジョブ5
]

In [6]:
# コスト
# 横軸はマシンm、縦軸はジョブi
C = [
     [1, 2, 3], # ジョブ1
     [1, 2, 1], # ジョブ2
     [2, 3, 1], # ジョブ3
     [3, 1, 2], # ジョブ4
     [2, 1, 1]  # ジョブ5
]

In [7]:
# リリース日
R = [2, 1, 3, 4, 5]
# 締切日
D = [5, 8, 10, 12, 10]

In [8]:
# ジョブiがマシンmに割り当てられているかを表すバイナリ変数x_im
# pulpの変数として宣言
# (17) (19) におけるx_i_m ∈ {0, 1} が実質定義される
x = [[pulp.LpVariable('x_{}_{}'.format(i, m), cat='Binary') for m in range(M)] for i in range(I)]

In [9]:
# ジョブの前後関係
y = [[pulp.LpVariable('y_{}_{}'.format(i, j), cat='Binary') for j in range(I)] for i in range(I)]

In [10]:
# 各マシンmでのジョブiの開始日ts_i
# (17) (19) におけるts_i >= 0 が実質定義される
ts = [pulp.LpVariable('ts_{}'.format(i), lowBound=0, cat='Integer') for i in range(I)]

In [11]:
# 目的関数(objective function)
obj = pulp.lpSum([C[i][m] * x[i][m] for i in range(I) for m in range(M)])

In [12]:
# 目的関数を追加
prob += obj

In [13]:
# 制約条件を追加 (19)
# ジョブに設定されたリリース日以降にジョブを開始 (10)
for i in range(I):
    prob += ts[i] >= R[i]

# ジョブに設定された締切日に間に合うようにジョブを開始 (11)
for i in range(I):
    prob += ts[i] <= D[i] - pulp.lpSum([P[i][m] * x[i][m] for m in range(M)])

# 各ジョブは1つのマシンのみで処理 (12)
for i in range(I):
    prob += pulp.lpSum([x[i][m] for m in range(M)]) == 1

# (13)
# j > i
for i in range(I):
    for j in range(i + 1, I):
        for m in range(M):
            prob += y[i][j] + y[j][i] >= x[i][m] + x[j][m] - 1

# (14)
U = sum(max(P[i]) for m in range(M) for i in range(I))
for i in range(I):
    for j in range(I):
        if i != j:
            pass
            prob += ts[j] >= ts[i] + pulp.lpSum([P[i][m] * x[i][m] for m in range(M)]) - U * (1 - y[i][j])

# (15)
# j > i
for i in range(I):
    for j in range(i + 1, I):
        prob += y[i][j] + y[j][i] <= 1

# (16)
# j > i, m != n
for i in range(I):
    for j in range(i + 1, I):
        for m in range(M):
            for n in range(M):
                if m != n:
                    prob += y[i][j] + y[j][i] + x[i][m] + x[j][n] <= 2

In [14]:
# 最終的な問題の定義を表示
prob

MILP:
MINIMIZE
1*x_0_0 + 2*x_0_1 + 3*x_0_2 + 1*x_1_0 + 2*x_1_1 + 1*x_1_2 + 2*x_2_0 + 3*x_2_1 + 1*x_2_2 + 3*x_3_0 + 1*x_3_1 + 2*x_3_2 + 2*x_4_0 + 1*x_4_1 + 1*x_4_2 + 0
SUBJECT TO
_C1: ts_0 >= 2

_C2: ts_1 >= 1

_C3: ts_2 >= 3

_C4: ts_3 >= 4

_C5: ts_4 >= 5

_C6: ts_0 + 8 x_0_0 + 3 x_0_1 + 6 x_0_2 <= 5

_C7: ts_1 + 5 x_1_0 + 3 x_1_1 + 10 x_1_2 <= 8

_C8: ts_2 + 9 x_2_0 + 4 x_2_1 + 6 x_2_2 <= 10

_C9: ts_3 + 4 x_3_0 + 9 x_3_1 + 11 x_3_2 <= 12

_C10: ts_4 + 9 x_4_0 + 4 x_4_1 + 4 x_4_2 <= 10

_C11: x_0_0 + x_0_1 + x_0_2 = 1

_C12: x_1_0 + x_1_1 + x_1_2 = 1

_C13: x_2_0 + x_2_1 + x_2_2 = 1

_C14: x_3_0 + x_3_1 + x_3_2 = 1

_C15: x_4_0 + x_4_1 + x_4_2 = 1

_C16: - x_0_0 - x_1_0 + y_0_1 + y_1_0 >= -1

_C17: - x_0_1 - x_1_1 + y_0_1 + y_1_0 >= -1

_C18: - x_0_2 - x_1_2 + y_0_1 + y_1_0 >= -1

_C19: - x_0_0 - x_2_0 + y_0_2 + y_2_0 >= -1

_C20: - x_0_1 - x_2_1 + y_0_2 + y_2_0 >= -1

_C21: - x_0_2 - x_2_2 + y_0_2 + y_2_0 >= -1

_C22: - x_0_0 - x_3_0 + y_0_3 + y_3_0 >= -1

_C23: - x_0_1 - x_3_1 + y_

In [15]:
# MILPを解く
status = prob.solve()

In [16]:
# 実行結果の状態を表示
# 解けていればOptimalと表示される
# それ以外の表示(Infeasible, Infeasible, Unbounded, Undefined)が出た場合には制約条件を見直す
pulp.LpStatus[status]

'Optimal'

In [17]:
# ランダム生成した処理時間PとコストCを表示
for i in range(I):
    for m in range(M):
        print('P_{}_{} = {},\tC_{}_{} = {}'.format(i, m, P[i][m], i, m, C[i][m]))

P_0_0 = 8,	C_0_0 = 1
P_0_1 = 3,	C_0_1 = 2
P_0_2 = 6,	C_0_2 = 3
P_1_0 = 5,	C_1_0 = 1
P_1_1 = 3,	C_1_1 = 2
P_1_2 = 10,	C_1_2 = 1
P_2_0 = 9,	C_2_0 = 2
P_2_1 = 4,	C_2_1 = 3
P_2_2 = 6,	C_2_2 = 1
P_3_0 = 4,	C_3_0 = 3
P_3_1 = 9,	C_3_1 = 1
P_3_2 = 11,	C_3_2 = 2
P_4_0 = 9,	C_4_0 = 2
P_4_1 = 4,	C_4_1 = 1
P_4_2 = 4,	C_4_2 = 1


In [18]:
# リリース日と締切日を表示
for i in range(I):
    print('R_{} = {},\tD_{} = {}'.format(i, R[i], i, D[i]))

R_0 = 2,	D_0 = 5
R_1 = 1,	D_1 = 8
R_2 = 3,	D_2 = 10
R_3 = 4,	D_3 = 12
R_4 = 5,	D_4 = 10


In [19]:
# ジョブごとに割り当てられたマシンと開始時間を表示
for i in range(I):
    m = 'na'
    for m in range(M):
        if x[i][m].value() == 1:
            print('i = {}:\tm = {},\tts = {}'.format(i, m, ts[i].value()))

i = 0:	m = 1,	ts = 2.0
i = 1:	m = 0,	ts = 1.0
i = 2:	m = 2,	ts = 3.0
i = 3:	m = 0,	ts = 6.0
i = 4:	m = 1,	ts = 5.0


In [20]:
# 割り当て結果のうち最も遅い終了時間を取得
t_max = 0
for i in range(I):
    for m in range(M):
        if x[i][m].value() == 1:
            t_max = max(t_max, int(ts[i].value() + P[i][m]))
# ガントチャートをm行t列で生成
time_table = [['na' for ts in range(t_max)] for m in range(M)]
# ガントチャートに割り当て結果を追加
for i in range(I):
    m = 'na'
    # マシンを取得
    for m in range(M):
        if x[i][m].value() == 1:
            # 開始時間を取得
            ts_ = int(ts[i].value())
            # 処理時間を取得
            P_ = P[i][m]
            # テーブルに追加
            for t in range(ts_, ts_ + P_):
                time_table[m][t] = 'j' + str(i)

In [21]:
# ガントチャートを表示
import pandas as pd
df = pd.DataFrame(time_table,
                  index=['m' + str(m) for m in range(M)],
                  columns=['t' + str(t) for t in range(t_max)])
df

,t0,t1,t2,t3,t4,t5,t6,t7,t8,t9
m0,na,j1,j1,j1,j1,j1,j3,j3,j3,j3
m1,na,na,j0,j0,j0,j4,j4,j4,j4,na
m2,na,na,na,j2,j2,j2,j2,j2,j2,na
